In [3]:
import xarray as xr
import dask
from dask.distributed import Client, LocalCluster
import matplotlib.pyplot as plt
import numpy as np

In [4]:
client = Client()
client

/home/dyllanjr/anaconda3/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33441 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:37263 Dashboard: http://127.0.0.1:33441/status,Cluster Workers: 4 Cores: 12 Memory: 50.56 GB


In [110]:
ds = xr.open_dataset('monthly_era5_2018.nc',
                     chunks={'time': -1, 'latitude': 'auto', 'longitude': 'auto'})
#I should find a better chunk size, will need to use for time series data, roughly 1 million elements per chunk, which
#is about 1/2 a year of data for 1 variable, since I ended up with 18 variables, the ideal chunk size should be around 1/18th a year

In [112]:
ds

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 12)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.25 359.5 359.75
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 2018-01-01 2018-02-01 ... 2018-12-01
Data variables:
    t2m        (time, latitude, longitude) float32 dask.array<chunksize=(12, 721, 1440), meta=np.ndarray>
    tp         (time, latitude, longitude) float32 dask.array<chunksize=(12, 721, 1440), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2020-09-17 04:27:02 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [113]:
#Min monthly value for whole year
annual_min = ds.min(dim='time')
annual_min = annual_min.rename({'t2m':'annual_min_t2m', 'tp':'annual_min_tp'})

In [114]:
#Max monthly value for whole year
annual_max = ds.max(dim='time')
annual_max = annual_max.rename({'t2m':'annual_max_t2m', 'tp':'annual_max_tp'})

In [115]:
#Annual mean value
annual_mean = ds.mean(dim='time')
annual_mean = annual_mean.rename({'t2m':'annual_mean_t2m', 'tp':'annual_mean_tp'})

In [116]:
north_hemi = ds.where(ds.latitude > 0, drop = True)
south_hemi = ds.where(ds.latitude < 0, drop =  True)

def as_mask(month):
    #creates boolean mask for summer/winter months (april-sept), depending on hemisphere
    return (month >= 4) & (month <= 9)

def om_mask(month):
    #creates boolean mask for summer/winter months (oct-march), depending on hemisphere
    return (month <= 3) | (month >= 10)


In [117]:
#Min/max monthly value for summer months (april-sept) for Northern Hemi
north_hemi_summer = north_hemi.sel(time=as_mask(north_hemi['time.month']))

north_hemi_summer_min = north_hemi_summer.min(dim = 'time')
north_hemi_summer_min = north_hemi_summer_min.rename({'t2m': 'sum_min_t2m', 'tp': 'sum_min_tp'})

north_hemi_summer_max = north_hemi_summer.max(dim = 'time')
north_hemi_summer_max = north_hemi_summer_max.rename({'t2m': 'sum_max_t2m', 'tp': 'sum_max_tp'})



In [118]:
#Min/max monthly value for summer months (oct-march) for southern hemi
south_hemi_summer = south_hemi.sel(time = om_mask(south_hemi['time.month']))

south_hemi_summer_min = south_hemi_summer.min(dim = 'time')
south_hemi_summer_min = south_hemi_summer_min.rename({'t2m': 'sum_min_t2m', 'tp': 'sum_min_tp'})

south_hemi_summer_max = south_hemi_summer.max(dim = 'time')
south_hemi_summer_max = south_hemi_summer_max.rename({'t2m': 'sum_max_t2m', 'tp': 'sum_max_tp'})


In [119]:
#min/max monthly value for winter months (oct-march) for Northern hemi
north_hemi_winter = north_hemi.sel(time=om_mask(north_hemi['time.month']))

north_hemi_winter_min = north_hemi_winter.min(dim = 'time')
north_hemi_winter_min = north_hemi_winter_min.rename({'t2m': 'win_min_t2m', 'tp': 'win_min_tp'})

north_hemi_winter_max = north_hemi_winter.max(dim = 'time')
north_hemi_winter_max = north_hemi_winter_max.rename({'t2m': 'win_max_t2m', 'tp': 'win_max_tp'})


In [120]:
#min/max monthly value for winter months (april-sept) for southern hemi
south_hemi_winter = south_hemi.sel(time = as_mask(south_hemi['time.month']))

south_hemi_winter_min = south_hemi_winter.min(dim = 'time')
south_hemi_winter_min = south_hemi_winter_min.rename({'t2m': 'win_min_t2m', 'tp': 'win_min_tp'})

south_hemi_winter_max = south_hemi_winter.max(dim = 'time')
south_hemi_winter_max = south_hemi_winter_max.rename({'t2m': 'win_max_t2m', 'tp': 'win_max_tp'})


In [121]:
#mean summer months MODIFIED VARIABLE NAMES FOR TESTING
north_hemi_summer_mean = north_hemi_summer.mean(dim = 'time')
north_hemi_summer_mean = north_hemi_summer_mean.rename({'t2m': 'sum_mean_t2m', 'tp': 'sum_mean_tp'})

south_hemi_summer_mean = south_hemi_summer.mean(dim = 'time')
south_hemi_summer_mean = south_hemi_summer_mean.rename({'t2m': 'sum_mean_t2m', 'tp': 'sum_mean_tp'})


In [122]:
#mean winter months
north_hemi_winter_mean = north_hemi_winter.mean(dim = 'time')
north_hemi_winter_mean = north_hemi_winter_mean.rename({'t2m': 'win_mean_t2m', 'tp': 'win_mean_tp'})


south_hemi_winter_mean = south_hemi_winter.mean(dim = 'time')
south_hemi_winter_mean = south_hemi_winter_mean.rename({'t2m': 'win_mean_t2m', 'tp': 'win_mean_tp'})


In [123]:
summer_min = xr.combine_nested([[north_hemi_summer_min],[south_hemi_summer_min]], concat_dim=['latitude', 'longitude'])
summer_max = xr.combine_nested([[north_hemi_summer_max],[south_hemi_summer_max]], concat_dim=['latitude', 'longitude'])
summer_mean = xr.combine_nested([[north_hemi_summer_mean],[south_hemi_summer_mean]], concat_dim=['latitude', 'longitude'])

winter_min = xr.combine_nested([[north_hemi_winter_min],[south_hemi_winter_min]], concat_dim=['latitude', 'longitude'])
winter_max = xr.combine_nested([[north_hemi_winter_max],[south_hemi_winter_max]], concat_dim=['latitude', 'longitude'])
winter_mean = xr.combine_nested([[north_hemi_winter_mean],[south_hemi_winter_mean]], concat_dim=['latitude', 'longitude'])

In [124]:
# Cols: annual_min, annual_max, annual_mean, 
#       summer_min, summer_max, summer_mean, 
#       winter_min, winter_max, winter_mean

# Rows: (lat, lon) should end up being 1,038,240 rows of data (721X1440)

In [125]:
dset = xr.combine_by_coords([annual_min, annual_max, annual_mean, summer_min, summer_max, summer_mean, winter_min, winter_max, winter_mean])

In [126]:
dset

<xarray.Dataset>
Dimensions:          (latitude: 721, longitude: 1440)
Coordinates:
  * latitude         (latitude) float64 -90.0 -89.75 -89.5 ... 89.5 89.75 90.0
  * longitude        (longitude) float32 0.0 0.25 0.5 ... 359.25 359.5 359.75
Data variables:
    annual_max_t2m   (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    annual_max_tp    (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    annual_mean_t2m  (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    annual_mean_tp   (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    annual_min_t2m   (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    annual_min_tp    (latitude, longitude) float32 dask.array<chunksize=(721, 1440), meta=np.ndarray>
    sum_max_t2m      (latitude, longitude) float32 dask.array<chunksize=(361, 1440), meta=np.ndarray>
    sum_max_tp       (latitude, longitude) float32 dask.array<chunksize=(361, 1440), meta=np.ndarray>
    sum_mean_t2m     (latitude, longitude) float32 dask.array<chunksize=(361, 1440), meta=np.ndarray>
    sum_mean_tp      (latitude, longitude) float32 dask.array<chunksize=(361, 1440), meta=np.ndarray>
    sum_min_t2m      (latitude, longitude) float32 dask.array<chunksize=(361, 1440), meta=np.ndarray>
    sum_min_tp       (latitude, longitude) float32 dask.array<chunksize=(361, 1440), meta=np.ndarray>
    win_max_t2m      (latitude, longitude) float32 dask.array<chunksize=(361, 1440), meta=np.ndarray>
    win_max_tp       (latitude, longitude) float32 dask.array<chunksize=(361, 1440), meta=np.ndarray>
    win_mean_t2m     (latitude, longitude) float32 dask.array<chunksize=(361, 1440), meta=np.ndarray>
    win_mean_tp      (latitude, longitude) float32 dask.array<chunksize=(361, 1440), meta=np.ndarray>
    win_min_t2m      (latitude, longitude) float32 dask.array<chunksize=(361, 1440), meta=np.ndarray>
    win_min_tp       (latitude, longitude) float32 dask.array<chunksize=(361, 1440), meta=np.ndarray>

In [127]:
dset.to_netcdf('era5_2018_prepared.nc')